In [1]:
import pandas as pd

import time
import numpy as np
import math
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torch import nn

import datetime as dt

from QLSTM import SequenceDataset

# import yfinance as yf
from pandas_datareader import data as pdr
# yf.pdr_override()

from itertools import product

import scienceplots

from QLSTM import ShallowRegressionLSTM

# Calculate the RMSE for the train and test data
from sklearn.metrics import mean_squared_error

# Define hyperparameter grid
batch_sizes = [5]
sequence_lengths = [1, 2, 3, 4, 5]
learning_rates = [0.01, 0.001]
hidden_units = [16, 32]
num_epochs_list = [20, 50]

# Generate all combinations of hyperparameters
param_grid = product(batch_sizes, sequence_lengths, learning_rates, hidden_units, num_epochs_list)

# Load the data
X = pd.read_csv("X.csv", index_col=0)
X

,MSFT,GOOGL,DEXJPUS,DEXUSUK,DJIA,VIXCLS,AAPL_1DT,AAPL_3DT,AAPL_6DT,AAPL_12DT,...,sia_cleaned_Comment_tokens_joined_lemmatized_pos,sia_cleaned_Comment_tokens_joined_lemmatized_compound,sia_cleaned_title_self_tokens_joined_lemmatized_neg,sia_cleaned_title_self_tokens_joined_lemmatized_neu,sia_cleaned_title_self_tokens_joined_lemmatized_pos,sia_cleaned_title_self_tokens_joined_lemmatized_compound,negative_finbert_comment,neutral_finbert_comment,positive_finbert_comment,finbert_compound
2019-12-11,4.977415,4.205386,4.688316,0.275812,10.236787,2.707383,0.008493,0.000221,0.042705,0.033765,...,0.144750,0.252900,0.035250,0.759500,0.205000,0.914275,0.300685,0.635741,0.063574,-0.300685
2019-12-12,4.987516,4.208535,4.692815,0.272543,10.244665,2.634762,0.002545,0.016866,0.036463,0.018928,...,0.133571,0.489700,0.028143,0.685714,0.143143,0.484814,0.077541,0.784491,0.137968,-0.077541
2019-12-13,4.995899,4.207333,4.693913,0.288856,10.244783,2.536075,0.013501,0.024540,0.035400,0.040269,...,0.100667,0.169000,0.056000,0.394833,0.049167,-0.120667,0.182847,0.756541,0.060613,-0.182847
2019-12-16,5.002349,4.217549,4.697202,0.287432,10.248349,2.496506,0.016973,0.033020,0.033241,0.043900,...,0.146500,0.246837,0.053000,0.736125,0.210875,0.646775,0.264257,0.644742,0.091001,-0.264257
2019-12-17,4.996934,4.213270,4.696107,0.271248,10.249456,2.508786,0.001963,0.032438,0.049304,0.048068,...,0.103286,0.114757,0.094429,0.688857,0.217000,0.640243,0.119574,0.825515,0.054911,-0.119574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-26,5.450865,4.584228,4.972933,0.067939,10.284004,3.473828,0.002258,-0.019377,0.000464,-0.024180,...,0.140333,0.027250,0.070667,0.818000,0.111333,0.437117,0.324316,0.595453,0.080231,-0.324316
2022-09-27,5.446476,4.577379,4.974732,0.072600,10.279695,3.484312,0.006545,-0.006437,-0.017764,-0.036299,...,0.237727,0.489700,0.082818,0.662273,0.164091,0.262518,0.122306,0.798727,0.078967,-0.122306
2022-09-28,5.465995,4.603197,4.970854,0.079920,10.298355,3.407179,-0.012732,-0.003930,-0.046041,-0.086817,...,0.176500,0.474720,0.107600,0.583700,0.108700,0.101580,0.339672,0.579781,0.080548,-0.339672
2022-09-29,5.451075,4.576558,4.972933,0.099664,10.282801,3.460723,-0.050366,-0.056554,-0.075931,-0.076712,...,0.150600,0.423050,0.087200,0.591000,0.121900,0.445760,0.277046,0.666928,0.056026,-0.277046


In [12]:
X.columns

Index(['MSFT', 'GOOGL', 'DEXJPUS', 'DEXUSUK', 'DJIA', 'VIXCLS', 'AAPL_1DT',
       'AAPL_3DT', 'AAPL_6DT', 'AAPL_12DT', 'SMA_21', 'SMA_63', 'SMA_252',
       'EMA_10', 'EMA_30', 'EMA_200', 'RSI_10', 'RSI_30', 'RSI_200', 'apple',
       'sia_cleaned_Comment_tokens_joined_lemmatized_neg',
       'sia_cleaned_Comment_tokens_joined_lemmatized_neu',
       'sia_cleaned_Comment_tokens_joined_lemmatized_pos',
       'sia_cleaned_Comment_tokens_joined_lemmatized_compound',
       'sia_cleaned_title_self_tokens_joined_lemmatized_neg',
       'sia_cleaned_title_self_tokens_joined_lemmatized_neu',
       'sia_cleaned_title_self_tokens_joined_lemmatized_pos',
       'sia_cleaned_title_self_tokens_joined_lemmatized_compound',
       'negative_finbert_comment', 'neutral_finbert_comment',
       'positive_finbert_comment', 'finbert_compound'],
      dtype='object')

In [2]:
Y = pd.read_csv("Y.csv", index_col=0)
Y

,AAPL_pred
2019-12-11,4.183982
2019-12-12,4.186527
2019-12-13,4.200029
2019-12-16,4.217002
2019-12-17,4.218966
...,...
2022-09-26,5.003637
2022-09-27,5.010182
2022-09-28,4.997449
2022-09-29,4.947083


In [3]:
# Combine X and Y into a single DataFrame for compatibility with the provided pipeline
df = pd.concat([X, Y], axis=1)
df

,MSFT,GOOGL,DEXJPUS,DEXUSUK,DJIA,VIXCLS,AAPL_1DT,AAPL_3DT,AAPL_6DT,AAPL_12DT,...,sia_cleaned_Comment_tokens_joined_lemmatized_compound,sia_cleaned_title_self_tokens_joined_lemmatized_neg,sia_cleaned_title_self_tokens_joined_lemmatized_neu,sia_cleaned_title_self_tokens_joined_lemmatized_pos,sia_cleaned_title_self_tokens_joined_lemmatized_compound,negative_finbert_comment,neutral_finbert_comment,positive_finbert_comment,finbert_compound,AAPL_pred
2019-12-11,4.977415,4.205386,4.688316,0.275812,10.236787,2.707383,0.008493,0.000221,0.042705,0.033765,...,0.252900,0.035250,0.759500,0.205000,0.914275,0.300685,0.635741,0.063574,-0.300685,4.183982
2019-12-12,4.987516,4.208535,4.692815,0.272543,10.244665,2.634762,0.002545,0.016866,0.036463,0.018928,...,0.489700,0.028143,0.685714,0.143143,0.484814,0.077541,0.784491,0.137968,-0.077541,4.186527
2019-12-13,4.995899,4.207333,4.693913,0.288856,10.244783,2.536075,0.013501,0.024540,0.035400,0.040269,...,0.169000,0.056000,0.394833,0.049167,-0.120667,0.182847,0.756541,0.060613,-0.182847,4.200029
2019-12-16,5.002349,4.217549,4.697202,0.287432,10.248349,2.496506,0.016973,0.033020,0.033241,0.043900,...,0.246837,0.053000,0.736125,0.210875,0.646775,0.264257,0.644742,0.091001,-0.264257,4.217002
2019-12-17,4.996934,4.213270,4.696107,0.271248,10.249456,2.508786,0.001963,0.032438,0.049304,0.048068,...,0.114757,0.094429,0.688857,0.217000,0.640243,0.119574,0.825515,0.054911,-0.119574,4.218966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-26,5.450865,4.584228,4.972933,0.067939,10.284004,3.473828,0.002258,-0.019377,0.000464,-0.024180,...,0.027250,0.070667,0.818000,0.111333,0.437117,0.324316,0.595453,0.080231,-0.324316,5.003637
2022-09-27,5.446476,4.577379,4.974732,0.072600,10.279695,3.484312,0.006545,-0.006437,-0.017764,-0.036299,...,0.489700,0.082818,0.662273,0.164091,0.262518,0.122306,0.798727,0.078967,-0.122306,5.010182
2022-09-28,5.465995,4.603197,4.970854,0.079920,10.298355,3.407179,-0.012732,-0.003930,-0.046041,-0.086817,...,0.474720,0.107600,0.583700,0.108700,0.101580,0.339672,0.579781,0.080548,-0.339672,4.997449
2022-09-29,5.451075,4.576558,4.972933,0.099664,10.282801,3.460723,-0.050366,-0.056554,-0.075931,-0.076712,...,0.423050,0.087200,0.591000,0.121900,0.445760,0.277046,0.666928,0.056026,-0.277046,4.947083


In [4]:
def normalize(a, min_a=None, max_a=None):
    if min_a is None:
        min_a, max_a = np.min(a, axis=0), np.max(a, axis=0)
    return (a - min_a) / (max_a - min_a + 0.0001), min_a, max_a

def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")
    return avg_loss

def test_model(data_loader, model, loss_function): 
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")
    return avg_loss

def predict(data_loader, model):
    """Just like `test_loop` function but keep track of the outputs instead of the loss
    function.
    """
    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)
    
    return output

# Calculate the accuracy of the model
def accuracy(y, y_star):
    return np.mean(np.abs(y - y_star) < 0.1)

In [5]:
# Extract the columns for features and target
columns = X.columns.tolist()
target = Y.columns[0]
dataset = df.values

# Splitting the data into train and test
size = int(len(df) * 0.8)
df_train = dataset[:size].copy()
df_test = dataset[size:].copy()

# Reconstruct DataFrame after splitting
df_train = pd.DataFrame(df_train, columns=columns + [target])
df_test = pd.DataFrame(df_test, columns=columns + [target])

# Normalize the data
df_train, min_train, max_train = normalize(df_train)
df_test, _, _ = normalize(df_test, min_train, max_train)

# Track the best configuration for the stock used
best_params = None
best_test_rmse = float('inf')
best_test_accuracy = float('inf')

# Add sensitivity analysis dictionary
sensitivity_analysis = {}

Case 1: Classical LSTM

In [ ]:
# Iterate through each hyperparameter combination
for batch_size, sequence_length, learning_rate, num_hidden_units, num_epochs in param_grid:
    print(f"Evaluating configuration: batch_size={batch_size}, sequence_length={sequence_length}, "
          f"learning_rate={learning_rate}, hidden_units={num_hidden_units}, num_epochs={num_epochs}")
    
    train_dataset = SequenceDataset(
        df_train,
        target=target,
        features=columns,
        sequence_length=sequence_length
    )
    test_dataset = SequenceDataset(
        df_test,
        target=target,
        features=columns,
        sequence_length=sequence_length
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    X, y = next(iter(train_loader))

    print("Features shape:", X.shape)
    print("Target shape:", y.shape)
    
    model = ShallowRegressionLSTM(
        num_sensors=len(columns),
        hidden_units=num_hidden_units,
        num_layers=1
    )
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Count number of parameters
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Number of parameters: {num_params}")

    classical_loss_train = []
    classical_loss_test = []
    print("Untrained test\n--------")
    test_loss = test_model(test_loader, model, loss_function)
    
    for ix_epoch in range(num_epochs):
        print(f"Epoch {ix_epoch}\n---------")
        train_loss = train_model(train_loader, model, loss_function, optimizer=optimizer)
        test_loss = test_model(test_loader, model, loss_function)
        classical_loss_train.append(train_loss)
        classical_loss_test.append(test_loss)
        print("")

    train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_eval_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    ystar_col_Q = "Model forecast"
    df_train[ystar_col_Q] = predict(train_eval_loader, model).numpy()
    df_test[ystar_col_Q] = predict(test_eval_loader, model).numpy()

    plt.figure(figsize=(12, 7))
    plt.plot(range(len(df_train)), df_train[target], label="Real data")
    plt.plot(range(len(df_train)), df_train[ystar_col_Q], label="LSTM train prediction")
    plt.ylabel('Target')
    plt.xlabel('Time steps')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 7))
    plt.plot(range(len(df_test)), df_test[target], label="Real data")
    plt.plot(range(len(df_test)), df_test[ystar_col_Q], label="LSTM test prediction")
    plt.ylabel('Target')
    plt.xlabel('Time steps')
    plt.legend()
    plt.show()

    df_test.to_csv('Test_LSTM_results.csv', index=False)

    plt.figure(figsize=(12, 7))
    plt.plot(classical_loss_train, label='Train Loss')
    plt.plot(classical_loss_test, label='Test Loss')
    pd.DataFrame(classical_loss_train).to_csv('LSTM_loss.csv', index=False)
    plt.title('Loss over epochs')
    plt.legend()

    train_rmse = math.sqrt(mean_squared_error(df_train[target], df_train[ystar_col_Q]))
    test_rmse = math.sqrt(mean_squared_error(df_test[target], df_test[ystar_col_Q]))
    print(f"Train RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")

    train_accuracy = accuracy(df_train[target], df_train[ystar_col_Q])
    test_accuracy = accuracy(df_test[target], df_test[ystar_col_Q])
    print(f"Train accuracy: {train_accuracy}")
    print(f"Test accuracy: {test_accuracy}")

    # Track sequence length sensitivity
    if sequence_length not in sensitivity_analysis:
        sensitivity_analysis[sequence_length] = []
    sensitivity_analysis[sequence_length].append((test_rmse, test_accuracy))

    # Update the best parameters if necessary
    if test_rmse < best_test_rmse or (test_rmse == best_test_rmse and test_accuracy > best_test_accuracy):
        best_test_rmse = test_rmse
        best_test_accuracy = test_accuracy  # Track the accuracy of the best configuration
        best_params = {
            "batch_size": batch_size,
            "sequence_length": sequence_length,
            "learning_rate": learning_rate,
            "hidden_units": num_hidden_units,
            "num_epochs": num_epochs
        }
    
    print(f"Best configuration: {best_params} with test RMSE {best_test_rmse} and test accuracy {best_test_accuracy}")

Case 2: Quantum LSTM (QLSTM)

In [ ]:
# Iterate through each hyperparameter combination
for batch_size, sequence_length, learning_rate, num_hidden_units, num_epochs in param_grid:
    print(f"Evaluating configuration: batch_size={batch_size}, sequence_length={sequence_length}, "
          f"learning_rate={learning_rate}, hidden_units={num_hidden_units}, num_epochs={num_epochs}")

    train_dataset = SequenceDataset(
        df_train,
        target=target,
        features=columns,
        sequence_length=sequence_length
    )
    test_dataset = SequenceDataset(
        df_test,
        target=target,
        features=columns,
        sequence_length=sequence_length
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    X, y = next(iter(train_loader))
    
    print("Features shape:", X.shape)
    print("Target shape:", y.shape)
    
    Qmodel = QShallowRegressionLSTM(
        num_sensors=len(columns), 
        hidden_units=num_hidden_units, 
        n_qubits=7,
        n_qlayers=1
    )
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(Qmodel.parameters(), lr=learning_rate)
    
    # Count number of parameters
    num_params = sum(p.numel() for p in Qmodel.parameters() if p.requires_grad)
    print(f"Number of parameters: {num_params}")
    
    quantum_loss_train = []
    quantum_loss_test = []
    print("Untrained test\n--------")
    
    for ix_epoch in range(num_epochs):
        print(f"Epoch {ix_epoch}\n---------")
        start = time.time()
        train_loss = train_model(train_loader, Qmodel, loss_function, optimizer=optimizer)
        test_loss = test_model(test_loader, Qmodel, loss_function)
        end = time.time()
        print("Execution time", end - start)
        quantum_loss_train.append(train_loss)
        quantum_loss_test.append(test_loss)
        print("")
    
    train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_eval_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    ystar_col_Q = "Model forecast"
    df_train[ystar_col_Q] = predict(train_eval_loader, Qmodel).numpy()
    df_test[ystar_col_Q] = predict(test_eval_loader, Qmodel).numpy()
    
    plt.figure(figsize=(12, 7))
    plt.plot(range(len(df_train)), df_train[target], label="Real data")
    plt.plot(range(len(df_train)), df_train[ystar_col_Q], label="QLSTM train prediction")
    plt.ylabel('Target')
    plt.xlabel('Time steps')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 7))
    plt.plot(range(len(df_test)), df_test[target], label="Real data")
    plt.plot(range(len(df_test)), df_test[ystar_col_Q], label="QLSTM test prediction")
    plt.ylabel('Target')
    plt.xlabel('Time steps')
    plt.legend()
    plt.show()

    df_test.to_csv('SPY_QLSTM_test.csv', index=False)

    plt.figure(figsize=(12, 7))
    plt.plot(quantum_loss_train, label='quantum_loss_train')
    plt.plot(quantum_loss_test, label='quantum_loss_test')
    pd.DataFrame(quantum_loss_train).to_csv('QLSTM_loss.csv', index=False)
    plt.title('Train loss')
    plt.legend()
    
    # Calculate the RMSE for the train and test data
    train_rmse = math.sqrt(mean_squared_error(df_train[target], df_train[ystar_col_Q]))
    test_rmse = math.sqrt(mean_squared_error(df_test[target], df_test[ystar_col_Q]))
    print(f"Train RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")
    
    # Calculate the accuracy of the model
    train_accuracy = accuracy(df_train[target], df_train[ystar_col_Q])
    test_accuracy = accuracy(df_test[target], df_test[ystar_col_Q])
    print(f"Train accuracy: {train_accuracy}")
    print(f"Test accuracy: {test_accuracy}")
    
    # Track sequence length sensitivity
    if sequence_length not in sensitivity_analysis:
        sensitivity_analysis[sequence_length] = []
    sensitivity_analysis[sequence_length].append((test_rmse, test_accuracy))

    # Update the best parameters if necessary
    if test_rmse < best_test_rmse or (test_rmse == best_test_rmse and test_accuracy > best_test_accuracy):
        best_test_rmse = test_rmse
        best_test_accuracy = test_accuracy  # Track the accuracy of the best configuration
        best_params = {
            "batch_size": batch_size,
            "sequence_length": sequence_length,
            "learning_rate": learning_rate,
            "hidden_units": num_hidden_units,
            "num_epochs": num_epochs
        }
    
    print(f"Best configuration: {best_params} with test RMSE {best_test_rmse} and test accuracy {best_test_accuracy}")

Case 3: Noisy QLSTM

In [ ]:
# Iterate through each hyperparameter combination
for batch_size, sequence_length, learning_rate, num_hidden_units, num_epochs in param_grid:
    print(f"Evaluating configuration: batch_size={batch_size}, sequence_length={sequence_length}, "
          f"learning_rate={learning_rate}, hidden_units={num_hidden_units}, num_epochs={num_epochs}")

    train_dataset = SequenceDataset(
        df_train,
        target=target,
        features=columns,
        sequence_length=sequence_length
    )
    test_dataset = SequenceDataset(
        df_test,
        target=target,
        features=columns,
        sequence_length=sequence_length
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    X, y = next(iter(train_loader))
    
    print("Features shape:", X.shape)
    print("Target shape:", y.shape)
    
    Qmodel = QShallowRegressionLSTM(
        num_sensors=len(columns), 
        hidden_units=num_hidden_units, 
        n_qubits=7,
        n_qlayers=1
    )
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(Qmodel.parameters(), lr=learning_rate)
    
    # Count number of parameters
    num_params = sum(p.numel() for p in Qmodel.parameters() if p.requires_grad)
    print(f"Number of parameters: {num_params}")
    
    quantum_loss_train = []
    quantum_loss_test = []
    print("Untrained test\n--------")
    
    for ix_epoch in range(num_epochs):
        print(f"Epoch {ix_epoch}\n---------")
        start = time.time()
        train_loss = train_model(train_loader, Qmodel, loss_function, optimizer=optimizer)
        test_loss = test_model(test_loader, Qmodel, loss_function)
        end = time.time()
        print("Execution time", end - start)
        quantum_loss_train.append(train_loss)
        quantum_loss_test.append(test_loss)
        print("")
    
    train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_eval_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    ystar_col_Q = "Model forecast"
    df_train[ystar_col_Q] = predict(train_eval_loader, Qmodel).numpy()
    df_test[ystar_col_Q] = predict(test_eval_loader, Qmodel).numpy()
    
    plt.figure(figsize=(12, 7))
    plt.plot(range(len(df_train)), df_train[target], label="Real data")
    plt.plot(range(len(df_train)), df_train[ystar_col_Q], label="QLSTM-noisy train prediction")
    plt.ylabel('Target')
    plt.xlabel('Time steps')
    plt.legend()
    plt.show()

    plt.figure(figsize=(12, 7))
    plt.plot(range(len(df_test)), df_test[target], label="Real data")
    plt.plot(range(len(df_test)), df_test[ystar_col_Q], label="QLSTM-noisy test prediction")
    plt.ylabel('Target')
    plt.xlabel('Time steps')
    plt.legend()
    plt.show()

    df_test.to_csv('SPY_QLSTM-noisy_test.csv', index=False)

    plt.figure(figsize=(12, 7))
    plt.plot(quantum_loss_train, label='quantum_loss_train')
    plt.plot(quantum_loss_test, label='quantum_loss_test')
    pd.DataFrame(quantum_loss_train).to_csv('QLSTM-noisy_loss.csv', index=False)
    plt.title('Train loss')
    plt.legend()
    
    # Calculate the RMSE for the train and test data
    train_rmse = math.sqrt(mean_squared_error(df_train[target], df_train[ystar_col_Q]))
    test_rmse = math.sqrt(mean_squared_error(df_test[target], df_test[ystar_col_Q]))
    print(f"Train RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")
    
    # Calculate the accuracy of the model
    train_accuracy = accuracy(df_train[target], df_train[ystar_col_Q])
    test_accuracy = accuracy(df_test[target], df_test[ystar_col_Q])
    print(f"Train accuracy: {train_accuracy}")
    print(f"Test accuracy: {test_accuracy}")
    
    # Track sequence length sensitivity
    if sequence_length not in sensitivity_analysis:
        sensitivity_analysis[sequence_length] = []
    sensitivity_analysis[sequence_length].append((test_rmse, test_accuracy))

    # Update the best parameters if necessary
    if test_rmse < best_test_rmse or (test_rmse == best_test_rmse and test_accuracy > best_test_accuracy):
        best_test_rmse = test_rmse
        best_test_accuracy = test_accuracy  # Track the accuracy of the best configuration
        best_params = {
            "batch_size": batch_size,
            "sequence_length": sequence_length,
            "learning_rate": learning_rate,
            "hidden_units": num_hidden_units,
            "num_epochs": num_epochs
        }
    
    print(f"Best configuration: {best_params} with test RMSE {best_test_rmse} and test accuracy {best_test_accuracy}")